In [2]:
%matplotlib inline
import sys, os
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")
import pcse
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

This notebook was built with:
python version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)] 
PCSE version: 5.5.5


In [42]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations

In [55]:
# Set time period
start = datetime(2004, 1, 1)
end = datetime(2005, 12, 31)

# Create Point for Vancouver, BC
point = Point(49.2497, -123.1193)

# Get daily data for 2018
data = Daily(point, start, end)
data = data.fetch()
data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2004-01-01,3.2,1.4,5.0,0.0,41.0,NaN,NaN,NaN,NaN,NaN
2004-01-02,1.6,0.3,2.9,0.0,20.0,NaN,NaN,NaN,NaN,NaN
2004-01-03,-2.7,-5.7,0.3,0.0,41.0,94.0,19.3,NaN,1021.3,NaN
2004-01-04,-5.2,-7.1,-3.2,0.0,41.0,NaN,9.6,NaN,1033.5,NaN
2004-01-05,-5.3,-7.2,-3.4,2.0,41.0,NaN,3.3,NaN,1035.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2005-12-27,8.3,7.2,9.3,11.0,0.0,NaN,NaN,NaN,NaN,NaN
2005-12-28,8.1,6.4,9.7,10.2,0.0,NaN,NaN,NaN,NaN,NaN
2005-12-29,7.5,5.9,9.0,0.8,0.0,NaN,NaN,NaN,NaN,NaN


In [65]:
data.to_excel('data\meteo\wdata.xlsx', index=False)

In [12]:
from pcse.fileinput import CABOFileReader

cropfile = os.path.join(data_dir, 'crop', 'SUG0601.crop')
cropd = CABOFileReader(cropfile)

In [4]:
soilfile = os.path.join(data_dir, 'soil', 'ec3.soil')
soild = CABOFileReader(soilfile)

In [5]:
from pcse.util import WOFOST72SiteDataProvider
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)

Unknown parameter values provided to WOFOSTSiteDataProvider: {'CO2': 360}
{'IFUNRN': 0, 'NOTINF': 0, 'SSI': 0.0, 'SSMAX': 0.0, 'WAV': 10.0, 'SMLIM': 0.4}


In [13]:
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

In [7]:
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

!!python/object/new:pcse.fileinput.yaml_agro_loader.YAMLAgroManagementReader
listitems:
- 2006-01-01:
    CropCalendar:
      crop_end_date: 2006-10-20
      crop_end_type: harvest
      crop_name: sugar-beet
      crop_start_date: 2006-04-05
      crop_start_type: emergence
      max_duration: 300
      variety_name: sugar-beet-601
    StateEvents: null
    TimedEvents: null



In [64]:
from pcse.fileinput import ExcelWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'wdata.xlsx')
wdp = ExcelWeatherDataProvider(weatherfile)
print(wdp)

KeyError: None

In [32]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
wofsim = Wofost72_WLP_FD(parameters, data, agromanagement)

TraitError: The 'weatherdataprovider' trait of a Wofost72_WLP_FD instance expected a WeatherDataProvider or None, not the DataFrame             tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2020-01-01   9.4   6.8  12.0   0.0   0.0  245.0  19.1  68.0  1006.1   NaN
2020-01-02   5.0   3.1   6.8   0.6   0.0   85.0  18.5  61.0  1016.9   NaN
2020-01-03   7.6   4.4  10.8   4.9   0.0  123.0  28.5  55.0  1007.5   NaN
2020-01-04   7.3   5.6   8.9   2.5   0.0  185.0  21.8  54.0  1019.3   NaN
2020-01-05   5.2   3.3   7.0   1.4   0.0  137.0  23.5  72.0  1021.6   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2020-12-27   6.2   5.6   6.9   5.8   0.0   71.0  10.8   NaN  1019.1   NaN
2020-12-28   5.4   4.1   6.6   1.3   0.0  314.0  13.1  39.0  1020.9   NaN
2020-12-29   4.3   3.2   5.4   7.1   0.0   83.0  12.3  32.0  1023.9   NaN
2020-12-30   5.2   3.7   6.7  39.8   0.0   78.0  25.6 

In [15]:
wofsim.run_till_terminate()
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

,DVS,LAI,TAGP,TWSO,TWLV,TWST,TWRT,TRA,RD,SM,WWLOW
day,,,,,,,,,,,
2006-10-16,2.0,1.048787,21557.705173,14410.185356,3408.824045,3738.695772,2805.686837,0.052878,120.0,0.283340,34.000845
2006-10-17,2.0,1.055901,21628.846417,14474.212476,3412.381108,3742.252834,2807.138699,0.030516,120.0,0.282876,33.945161
2006-10-18,2.0,1.049414,21658.771700,14501.145230,3413.877372,3743.749098,2807.749419,0.030184,120.0,0.284605,34.152545
2006-10-19,2.0,1.052133,21685.964928,14525.619135,3415.237033,3745.108759,2808.304383,0.043053,120.0,0.284840,34.180768
2006-10-20,2.0,1.043584,21717.141971,14553.678474,3416.795885,3746.667611,2808.940650,0.039061,120.0,0.284271,34.112520
